## Download model

It's important that your model have this directory structure for Triton Inference Server to be able to load it. [Read more about the directory structure that Triton expects](https://docs.nvidia.com/deeplearning/triton-inference-server/user-guide/docs/model_repository.html).

In [4]:
!pip install azure-storage-blob

In [6]:
import os
import sys
from pathlib import Path
from src.model_utils import download_triton_models, delete_triton_models

prefix = Path(".")
download_triton_models(prefix)

successfully downloaded model: densenet_onnx


## Register model

The registered model should follow the Triton specified model folder structure for Triton Inference Server to be able to load it.

In [9]:
subscripton = "13e50845-67bc-4ac5-94db-48d493a6d9e8"
resource_group = "chrjia-rg"
workspace = "chrjia-eastus2euap2"
model_name = "densenet-onnx"
service_name = "lite14"

In [10]:
!az ml model create -n $model_name -v 1 -l models -g $resource_group -w $workspace --subscription $subscripton

Command group 'ml model' is experimental and under development. Reference and support levels: https://aka.ms/CLI_refstatus
Uploading models: 100%|█████████████████████████| 1/1 [00:00<00:00, 4911.36it/s]
{
  "creation_context": {
    "created_at": "2021-05-18T05:25:45.465914+00:00",
    "created_by": "Arzoo Aneja (ZEN3 INFOSOLUTIONS AMERICA INC)",
    "created_by_type": "User",
    "last_modified_at": "2021-05-18T05:25:45.465914+00:00",
    "last_modified_by": "Arzoo Aneja (ZEN3 INFOSOLUTIONS AMERICA INC)",
    "last_modified_by_type": "User"
  },
  "datastore": "azureml:/subscriptions/13e50845-67bc-4ac5-94db-48d493a6d9e8/resourceGroups/chrjia-rg/providers/Microsoft.MachineLearningServices/workspaces/chrjia-eastus2euap2/datastores/workspaceblobstore",
  "flavors": {},
  "id": "azureml:/subscriptions/13e50845-67bc-4ac5-94db-48d493a6d9e8/resourceGroups/chrjia-rg/providers/Microsoft.MachineLearningServices/workspaces/chrjia-eastus2euap2/models/densenet-onnx/versions/1",
  "name": "densene

In [11]:
!az ml model show -n $model_name -v 1 -g $resource_group -w $workspace --subscription $subscripton

Command group 'ml model' is experimental and under development. Reference and support levels: https://aka.ms/CLI_refstatus
{
  "creation_context": {
    "created_at": "2021-05-18T05:25:45.465914+00:00",
    "created_by": "Arzoo Aneja (ZEN3 INFOSOLUTIONS AMERICA INC)",
    "created_by_type": "User",
    "last_modified_at": "2021-05-18T05:25:45.465914+00:00",
    "last_modified_by": "Arzoo Aneja (ZEN3 INFOSOLUTIONS AMERICA INC)",
    "last_modified_by_type": "User"
  },
  "datastore": "azureml:/subscriptions/13e50845-67bc-4ac5-94db-48d493a6d9e8/resourceGroups/chrjia-rg/providers/Microsoft.MachineLearningServices/workspaces/chrjia-eastus2euap2/datastores/workspaceblobstore",
  "flavors": {},
  "id": "azureml:/subscriptions/13e50845-67bc-4ac5-94db-48d493a6d9e8/resourceGroups/chrjia-rg/providers/Microsoft.MachineLearningServices/workspaces/chrjia-eastus2euap2/models/densenet-onnx/versions/1",
  "name": "densenet-onnx",
  "path": "az-ml-artifacts/57396fa60078750da5128ce974a4a65f/models",
  "

## Create endpoint

Deploy to a pre created AKS compute

In [17]:
!az ml endpoint create -g $resource_group -w $workspace -n $service_name -f deployment.yml

Command group 'ml endpoint' is experimental and under development. Reference and support levels: https://aka.ms/CLI_refstatus
Code: AuthorizationFailed
Message: The client 'v-arzooaneja@microsoft.com' with object id '518cdffb-43cb-46ba-b849-f6c1e2225378' does not have authorization to perform action 'Microsoft.MachineLearningServices/workspaces/datastores/read' over scope '/subscriptions/bac420ed-c6fc-4a05-8ac1-8c0c52da1d6e/resourceGroups/chrjia-rg/providers/Microsoft.MachineLearningServices/workspaces/chrjia-eastus2euap2' or the scope is invalid. If access was recently granted, please refresh your credentials.


In [ ]:
!az ml endpoint show -g $resource_group -w $workspace -n $service_name

## Test Webservice

Get scoring URI and auth token

In [ ]:
!az ml endpoint show -n $service_name --query "scoring_uri"

In [ ]:
!az ml endpoint list-keys -g $resource_group -w $workspace -n $service_name

In [ ]:
import requests

service_key = "service_key"
headers = {}
headers["Authorization"] = f"Bearer {service_key}"

# Check the state of server.
service_url = "service_url"
requests.get(f"{service_url}/v2/health/ready", headers=headers)

In [ ]:
# Check the status of model.
resp = requests.post(f"{service_url}/v2/repository/index", headers=headers)
print(resp.text)

In [ ]:
# Check metadata of model for inference 
resp = requests.get(f"{service_url}/v2/models/densenet_onnx", headers=headers)
print(resp.text)